<a href="https://colab.research.google.com/github/chu-ise/378A-2022/blob/main/exam/midterm_2022.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Midterm 378A-2022

In [ ]:
name = input("What is your name? ")
sid = input("What is your student ID? ")
print("Name: " + name + "\nStudent ID: " + sid)

What is your name? Your Name
What is your student ID? Your ID
Name: Your Name
Student ID: Your ID


## Problem specification

1. 의사록 다운로드
2. 문장 단위로 분리
3. 토크나이징 (형태소 분석 후 명사 추출)
4. 각 문서별 불확실성 단어 추출 후 카운트
5. 연도별 불확실성 단어 발생빈도 시계열 그래프 생성

> 불확실성 단어 리스트:
> 
> ['불확실', '위험', '경제', '적자', '국회', '입법', '청와대', '정부', '규제']


In [1]:
%%capture
%pip install ekorpkit[tokenize]==0.1.17

In [2]:
from ekorpkit.io.download import gdrive_download
id = "1uDxxBNZ-qZscPjdEam0QEjTdKcFUBMuy"

data_file = "bok_minutes.csv"
gdrive_download(file_id=id, local_path=data_file)

[bok_minutes.csv]: 100%|██████████| 10.1M/10.1M [00:00<00:00, 19.2MB/s]


In [3]:
import pandas as pd
df = pd.read_csv(data_file)
df.head()

,id,filename,mdate,rdate,section,text
0,BOK_20181130_20181218_S1,BOK_20181130_20181218,2018-11-30 10:00:00,2018-12-18 16:00:00,Economic Situation,일부 위원은 관련부서에서 지난 3/4분기 중 유로지역 경제성장 부진을 자동차 관련 ...
1,BOK_20181130_20181218_S2,BOK_20181130_20181218,2018-11-30 10:00:00,2018-12-18 16:00:00,Foreign Currency,일부 위원은 그동안 글로벌펀드와 패시브펀드의 규모가 크게 확대되어 우리나라 자본유출...
2,BOK_20181130_20181218_S3,BOK_20181130_20181218,2018-11-30 10:00:00,2018-12-18 16:00:00,Financial Markets,"일부 위원은 현재 대기업들이 전반적으로는 문제가 없지만, 건설 조선업 등에 속하는 ..."
3,BOK_20181130_20181218_S4,BOK_20181130_20181218,2018-11-30 10:00:00,2018-12-18 16:00:00,Monetary Policy,일부 위원은 최근 경기상황과 금융불균형 등을 고려할 때 확장적 재정정책의 필요성에는...
4,BOK_20181130_20181218_S5,BOK_20181130_20181218,2018-11-30 10:00:00,2018-12-18 16:00:00,Participants’ Views,일부 위원은 최근 실물경제 성장경로의 하방위험이 다소 커진 것으로 보이고 물가도 상...



## prepare tokenizers

In [4]:
from ekorpkit import eKonf
config_group='preprocessor/tokenizer=mecab'
cfg = eKonf.compose(config_group=config_group)
mecab = eKonf.instantiate(cfg)
print(mecab.pos('IMF가 추정한 우리나라의 GDP갭률은 금년에도 소폭의 마이너스(-)를 지속하고 있다.'))

[ekorpkit]: Initializing mecab...


['IMF/SL', '가/JKS', ' /SP', '추정/NNG', '한/XSA+ETM', ' /SP', '우리나라/NNG', '의/JKG', ' /SP', 'GDP/SL', '갭/NNG', '률/XSN', '은/JX', ' /SP', '금년/NNG', '에/JKB', '도/JX', ' /SP', '소폭/NNG', '의/JKG', ' /SP', '마이너스/NNG', '(/SSO', '-)/SY', '를/JKO', ' /SP', '지속/NNG', '하/XSV', '고/EC', ' /SP', '있/VX', '다/EF', './SF']


In [5]:
from ekorpkit.preprocessors.tokenizer import MecabTokenizer
mecab = MecabTokenizer()
uncertain_words = ['불확실', '위험', '경제', '적자', '국회', '입법', '청와대', '정부', '규제']
print(mecab.pos(' '.join(uncertain_words)))

[ekorpkit]: Initializing mecab...


['불확실/NNG', ' /SP', '위험/NNG', ' /SP', '경제/NNG', ' /SP', '적자/NNG', ' /SP', '국회/NNG', ' /SP', '입법/NNG', ' /SP', '청와대/NNP', ' /SP', '정부/NNG', ' /SP', '규제/NNG']


## Start your coding ...